<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: **SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Joaquín De Groote
- Nombre de alumno 2: Vicente Pinochet R.

### **Link de repositorio de GitHub:** [Insertar Enlace](https://github.com/Qajirr/MDS7202-Labs)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [3]:
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import pickle
set_config(transform_output="pandas")

In [4]:
# Inserte su código acá
# 1. Separar los datos
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=99)
val_data, test_data = train_test_split(temp_data, test_size=0.33, random_state=99)

In [5]:
# Definir la función para extraer día, mes y año
def extract_date_features(df):
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['day'] = df['day'].astype('category')
    df['month'] = df['month'].astype('category')
    df['year'] = df['year'].astype('category')
    return df.drop(columns='date')

# 2. Crear el transformer
date_transformer = FunctionTransformer(extract_date_features)

In [6]:
# Seleccionar las columnas categóricas y numéricas
categorical_columns = ['city', 'shop', 'day', 'month', 'year', 'brand',	'container', 'capacity']
numeric_columns = ['id', 'lat',	'long', 'pop', 'price']

# 3. Definir el transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(sparse_output=False), categorical_columns)
    ]
)
# Configurar salida en formato DataFrame
preprocessor.set_output(transform="pandas")

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['id', 'lat', 'long', 'pop', 'price']),
                                ('cat', OneHotEncoder(sparse_output=False),
                                 ['city', 'shop', 'day', 'month', 'year',
                                  'brand', 'container', 'capacity'])])

In [7]:
# 4. Crear el pipeline con el preprocesador y el DummyRegressor
pipeline_dummy = Pipeline(steps=[
    ('date to categorical', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

In [8]:
# 5. Entrenar el modelo
X_train = train_data.drop(columns='quantity')
y_train = train_data['quantity']
pipeline_dummy.fit(X_train, y_train)

X_val = val_data.drop(columns='quantity')
y_val = val_data['quantity']

# Predicciones
y_pred_dummy = pipeline_dummy.predict(X_val)

# Calcular MAE
mae_dummy = mean_absolute_error(y_val, y_pred_dummy)
print(f"MAE del DummyRegressor: {mae_dummy:.2f}")

MAE del DummyRegressor: 13735.82


MAE del DummyRegressor: 13735.82

In [9]:
# 6. Crear el pipeline con XGBRegressor
pipeline_xgb = Pipeline(steps=[
    ('date to categorical', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

# Entrenar el modelo
pipeline_xgb.fit(X_train, y_train)

# Predicciones
y_pred_xgb = pipeline_xgb.predict(X_val)

# Calcular MAE
mae_xgb = mean_absolute_error(y_val, y_pred_xgb)
print(f"MAE del XGBRegressor: {mae_xgb:.2f}")

MAE del XGBRegressor: 2595.22


MAE del XGBRegressor: 2595.22

In [10]:
# 7. Guardar el DummyRegressor
with open('dummy_regressor.pkl', 'wb') as f:
    pickle.dump(pipeline_dummy, f)

# Guardar el XGBRegressor
with open('xgb_regressor.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [11]:
# Inserte su código acá
# Obtener nombres de las columnas transformadas
feature_names = pipeline_xgb.named_steps['preprocessor'].get_feature_names_out()
price_index = list(feature_names).index('num__price')

# Configurar las restricciones monotónicas
relacion_precio = (0,) * len(feature_names)  # Inicializamos relación para cada variable en 0
relacion_precio = list(relacion_precio)
relacion_precio[price_index] = -1  # Forzamos la relación negativa entre precio y cantidad (monótona decreciente)
relacion_precio = tuple(relacion_precio)

pipeline_xgb_monotonic = Pipeline(steps=[
    ('date to categorical', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=relacion_precio))
])

# Entrenar el modelo
pipeline_xgb_monotonic.fit(X_train, y_train)

Pipeline(steps=[('date to categorical',
                 FunctionTransformer(func=<function extract_date_features at 0x00000219CE63FC40>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['id', 'lat', 'long', 'pop',
                                                   'price']),
                                                 ('cat',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['city', 'shop', 'day',
                                                   'month', 'year', 'brand',
                                                   'container',
                                                   'capacity'])])),
                ('regressor',...
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=(0, 0, 0, 0, -1, 0, 0, 0, 0,
                                                    0, 0, 0, 0, 0, 0, 0, 0, 0,
                                                    0, 0, 0, 0, 0, 0, 0, 0, 0,
                                                    0, 0, 0, ...),
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [12]:
# Predicciones con el nuevo modelo
y_pred_xgb_monotonic = pipeline_xgb_monotonic.predict(X_val)

# Calcular el MAE
mae_xgb_monotonic = mean_absolute_error(y_val, y_pred_xgb_monotonic)
print(f"MAE del XGBRegressor con constraints: {mae_xgb_monotonic:.2f}")


MAE del XGBRegressor con constraints: 2660.86


MAE del XGBRegressor con constraints: 2660.86

In [13]:
# Guardar el modelo con restricciones en un archivo .pkl
with open('xgb_regressor_monotonic.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb_monotonic, f)

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [19]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Definir la función de optimización
def objective(trial):
    # Espacio de búsqueda de hiperparámetros
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0, 1)
    reg_lambda = trial.suggest_float("reg_lambda", 0, 1)
    min_frequency = trial.suggest_float("min_frequency", 0.0, 1.0)
    
    # Pipeline con XGBRegressor y optimización de OneHotEncoder
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_columns),
            ('cat', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), categorical_columns)
        ]
    )
    
    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=99
    )
    
    pipeline = Pipeline(steps=[
        ('date to categorical', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # Entrenar el pipeline con los datos de entrenamiento
    pipeline.fit(X_train, y_train)
    
    # Predicción en el conjunto de validación
    y_pred = pipeline.predict(X_val)
    
    # Calcular el MAE en el conjunto de validación
    mae = mean_absolute_error(y_val, y_pred)
    
    # Almacenar el pipeline entrenado en el trial si es el mejor hasta ahora
    trial.set_user_attr("best_pipeline", pipeline)
    
    return mae


In [20]:
# Configuración del estudio con Optuna
sampler = TPESampler(seed=99)
study = optuna.create_study(direction="minimize", sampler=sampler)

# Ejecutar la optimización con un tiempo límite de 5 minutos (300 segundos)
study.optimize(objective, timeout=300)

# Mostrar los resultados de la optimización
best_trial = study.best_trial
print(f"Número de trials: {len(study.trials)}")
print(f"Mejor MAE: {best_trial.value}")
print(f"Mejores hiperparámetros: {best_trial.params}")
# Guardar el mejor pipeline encontrado
best_pipeline = best_trial.user_attrs["best_pipeline"]

# Guardar el modelo en un archivo .pkl
with open('best_xgb_pipeline.pkl', 'wb') as f:
    pickle.dump(best_pipeline, f)


Número de trials: 348
Mejor MAE: 2114.4488751929657
Mejores hiperparámetros: {'learning_rate': 0.08146916634099562, 'n_estimators': 960, 'max_depth': 6, 'max_leaves': 61, 'min_child_weight': 2, 'reg_alpha': 0.044832010859262916, 'reg_lambda': 0.934815959479738, 'min_frequency': 0.04513572037753127}


learning_rate: controla la tasa de aprendizaje del modelo. Un valor pequeño hace que el modelo aprenda más lentamente pero con mayor precisión.

n_estimators: el número de árboles en el modelo. Un valor mayor permite más árboles, lo que aumenta la capacidad del modelo, pero también puede causar sobreajuste.

max_depth: la profundidad máxima de los árboles. Controla la complejidad del modelo. Un valor mayor permite árboles más profundos pero incrementa el riesgo de sobreajuste.

max_leaves: número máximo de hojas por árbol. Controla la complejidad de los árboles, permitiendo que se formen más hojas para capturar más patrones.

min_child_weight: el peso mínimo de una hoja, lo que afecta el número mínimo de muestras necesarias para dividir un nodo. Valores más altos restringen la división de nodos.

reg_alpha: regularización L1 (Lasso), que impone una penalización en las características que no son importantes para forzar la reducción de coeficientes.

reg_lambda: regularización L2 (Ridge), que penaliza grandes coeficientes y ayuda a controlar el sobreajuste.

min_frequency: en el OneHotEncoder, filtra categorías con frecuencias menores a este valor, lo que puede reducir el ruido causado por categorías poco frecuentes.




## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [24]:
!pip install optuna-integration[xgboost]


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [25]:
# Inserte su código acá
import optuna
from optuna.integration import XGBoostPruningCallback
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Redefinir la función objective con pruning
# Definir la función objective con el preprocesamiento adecuado
def objective(trial):
    # Espacio de búsqueda de hiperparámetros
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.05)
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 6)
    max_leaves = trial.suggest_int("max_leaves", 0, 50)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0, 1)
    reg_lambda = trial.suggest_float("reg_lambda", 0, 1)
    min_frequency = trial.suggest_float("min_frequency", 0.1, 0.5)
    
    # Pipeline con preprocesamiento
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_columns),
            ('cat', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), categorical_columns)
        ]
    )

    # Crear el modelo con XGBRegressor
    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=99,
        use_label_encoder=False,  # No usar LabelEncoder de XGBoost
        enable_categorical=False  # Se asegura que las categóricas sean numéricas ya
    )

    # Definir el pipeline
    pipeline = Pipeline(steps=[
        ('date to categorical', date_transformer),  # Para extraer day, month, year
        ('preprocessor', preprocessor),  # Preprocesar numéricas y categóricas
        ('regressor', model)  # XGBRegressor
    ])

    # Entrenar el pipeline
    pipeline.fit(X_train, y_train)

    # Predicción y cálculo del MAE en el conjunto de validación
    y_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    # Almacenar el mejor pipeline si es el mejor hasta ahora
    trial.set_user_attr("best_pipeline", pipeline)

    return mae



In [26]:
# Fijar el tiempo a 5 minutos y usar pruning
sampler = TPESampler(seed=99)
study = optuna.create_study(direction="minimize", sampler=sampler)

# Silenciar los prints y añadir barra de progreso
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Ejecutar la optimización con pruning y un tiempo límite de 5 minutos
study.optimize(objective, timeout=300, show_progress_bar=True)

# Mostrar los resultados de la optimización
best_trial = study.best_trial
print(f"Número de trials: {len(study.trials)}")
print(f"Mejor MAE: {best_trial.value}")
print(f"Mejores hiperparámetros: {best_trial.params}")


   0%|          | 00:00/05:00

[W 2024-10-24 12:37:25,974] Trial 0 failed with parameters: {'learning_rate': 0.03689114234523167, 'n_estimators': 514, 'max_depth': 6, 'max_leaves': 1, 'min_child_weight': 5, 'reg_alpha': 0.5656174196105306, 'reg_lambda': 0.29762249870394397, 'min_frequency': 0.04669572050695325} because of the following error: ValueError('DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:date: datetime64[ns], city: object, shop: object, brand: object, container: object, capacity: object, day: category, month: category, year: category').
Traceback (most recent call last):
  File "c:\Users\joaqu\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\joaqu\AppData\Local\Temp\ipykernel_16212\217477060.py", line 50, in objecti

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:date: datetime64[ns], city: object, shop: object, brand: object, container: object, capacity: object, day: category, month: category, year: category

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>